In [ ]:
import json
import os
from datetime import datetime

import pandas as pd
import plotly.express as px

from preepiseizures.src import Patient, Biosignal
from preepiseizures.temporal_analysis import temporal_hr_sync as ths


In [ ]:

patient_dict_ = json.load(open('patient_info.json', 'r'))
patient_list = list(patient_dict_.keys())
for patient in patient_list[30:40]:
    # print('patient: ', patient)
    #if 'temporal_shift' not in patient_dict_[patient].keys():
    #    continue
    # print(patient_dict_[patient])
    if not os.path.isfile(patient + '_hospital_hr.parquet'):
         continue
    if not os.path.isfile(patient + '_wearable_hr.parquet'):
         continue
    # continue
    print(patient, ' is good')
    patient_class = Patient.patient_class(patient)

    ## GET HR DATA
    if patient in ['VNVW', 'CNSV']:
        
        hr_wear = pd.read_parquet(patient + '_wearable_hr.parquet')
        hr_correct = hr_wear.rename(columns={'timestamp': 'datetime'})
        if patient == 'VNVW':
            hr_wear = Patient.correct_patient(hr_correct, patient)
        hr_hosp = pd.read_parquet(patient + '_hospital_hr.parquet')
    else:
        hr_wear = ths.get_all_hr_wearable(patient_class)
        hr_hosp = ths.get_all_hr_hospital(patient_class)
    if len(hr_hosp) < 1:
        continue
    hr_new = ths.process_hr(hr_wear)
    hr_new_2 = ths.process_hr(hr_hosp)
    df_all, df_w = ths.compute_correlation(hr_new_2, hr_new)
    temporal_shift = df_all['hr_w'].dropna().index[0] - hr_new.index[0]
    print('temporal shift: ', temporal_shift)
    df_all['old'] = hr_new['hr']

    fig = px.line(df_all, x=df_all.index, y=['hr_w', 'hr_h', 'old'], title=patient)
    fig.show()
    

    
    

In [ ]:
patient = 'YIVL'
patient_class = Patient.patient_class(patient)
patient_class.get_seizure_annotations()
## GET HR DATA
hr_wear = ths.get_all_hr_wearable(patient_class)
print(hr_wear.head(5))
hr_hosp = ths.get_all_hr_hospital(patient_class)
# hr_wear['datetime'] = hr_wear['timestamp']
# hr_wear = Patient.correct_patient(hr_wear, patient)
# hr_wear.index = hr_wear['datetime']
# hr_wear = hr_wear.drop(columns=['datetime'])
# hr_wear['timestamp'] = hr_wear.index

In [ ]:
hr_hosp

In [ ]:
hr_wear

In [ ]:
patient_class.id

In [ ]:
import biosppy as bp
def process_hr(hr_df):
    """
    Receives an hr dataframe with timestamps column and hr column.
    Resamples and processes the HR signal to have a clean and continuous signal.
    ----
    Parameters:
        hr_df: pd.DataFrame - Dataframe with hr data
    ----
    Returns:
        hr_df: pd.DataFrame - Dataframe with processed hr data
    """
    hr_df = hr_df.set_index('timestamp')
    # median filter

    hr_df = hr_df.resample('5S').median()
    hr_df = hr_df.interpolate(method='linear', limit_direction='both')
    hr_df = hr_df.dropna()
    # clean even more
    # hr_df = hr_df.resample('30S').median()
    hr_filt = bp.signals.tools.filter_signal(signal=hr_df['hr'].values, ftype='butter', band='lowpass', order=4, frequency=0.005, sampling_rate=0.2)['signal']
    hr_df['hr'] = hr_filt
    return hr_df


In [ ]:
hr_new = process_hr(hr_wear)#.iloc[:7200] # .loc['2020-1-7 17:59:00':'2020-1-7 03:59:59']
hr_new_2 = process_hr(hr_hosp)#.iloc[17200:30000]

In [ ]:
hr_clean = hr_wear.loc[hr_wear['hr']< 120]
fig = px.line(hr_clean, x='timestamp', y='hr', title=patient)
hr_new = process_hr(hr_clean)#.iloc[:7200] # .loc['2020-1-7 17:59:00':'2020-1-7 03:59:59']

fig.show()

In [ ]:
fig = px.line(hr_new, x=hr_new.index, y='hr', title=patient)
fig.add_scatter(x=hr_new_2.index, y=hr_new_2['hr'], mode='lines', name='hospital')
fig.show()

In [ ]:

df_all, df_w = ths.compute_correlation(hr_new_2, hr_new)
temporal_shift = df_all['hr_w'].dropna().index[0] - hr_new.index[0]
print('temporal shift: ', temporal_shift)

fig = px.line(df_all, x=df_all.index, y=['hr_w', 'hr_h'], title=patient)
fig.show()

df_old = pd.DataFrame({'hr_w':hr_new['hr'], 'hr_h':hr_new_2['hr']}) 
fig = px.line(df_old, x=df_old.index, y=['hr_w', 'hr_h'], title=patient + ' no temporal shift')
for i in range(len(patient_class.seizure_table)):
    fig.add_vline(x=patient_class.seizure_table['Timestamp'].iloc[i], line_width=3, line_dash="dash", line_color="green")
fig.show()

In [ ]:
df_old = pd.DataFrame({'hr_w':hr_new['hr'], 'hr_h':hr_new_2['hr']}) 
df_old['hr_w'] = df_old['hr_w'].shift(periods=1150, freq='1S')
fig = px.line(df_old, x=df_old.index, y=['hr_w', 'hr_h'], title=patient + ' manual temporal shift')
for i in range(len(patient_class.seizure_table)):
    fig.add_vline(x=patient_class.seizure_table['Timestamp'].iloc[i], line_width=3, line_dash="dash", line_color="green")
fig.show()

In [ ]:

df_old = pd.DataFrame({'hr_w':hr_new['hr'], 'hr_h':hr_new_2['hr']}) 
fig = px.line(df_old, x=df_old.index, y=['hr_w', 'hr_h'], title=patient + ' no temporal shift')
for i in range(len(patient_class.seizure_table)):
    fig.add_vline(x=patient_class.seizure_table['Timestamp'].iloc[i], line_width=3, line_dash="dash", line_color="green", 
                  name=patient_class.seizure_table['Focal / Generalisada'].iloc[i])
fig.show()

In [64]:
patient = 'YIVL'
patient_class = Patient.patient_class(patient)
patient_class.get_seizure_annotations()
## GET HR DATA
hr_wear = ths.get_all_hr_wearable(patient_class)
print(hr_wear.head(5))
hr_hosp = ths.get_all_hr_hospital(patient_class)

patient_class.seizure_table

   hr                  timestamp
0 NaN 2022-08-08 15:53:38.194146
1 NaN 2022-08-08 15:53:40.000146
2 NaN 2022-08-08 15:54:00.000146
3 NaN 2022-08-08 15:54:20.618146
4 NaN 2022-08-08 15:54:20.928146


,Crises,Data,Hora Clínica,Localização,lado,Focal / Generalisada,Fim,Sono/ Vigília,Tipo,Timestamp
0,Crise1,2022-08-10,01:09:27,FC e PO,L,FBTC,01:11:20,Sono,Hiperventilação e taquicardia e TC,2022-08-10 01:09:27
1,Crise2,2022-08-10,10:37:26,FC e PO,L,FBTC,10:39:19,Sono?,Hiperventilação e taquicardia e TC,2022-08-10 10:37:26
2,Crise3,2022-08-11,01:36:30,FC e PO,L,FBTC,01:38:23,Sono,Hiperventilação e taquicardia e TC,2022-08-11 01:36:30


In [62]:
start_time = hr_hosp['timestamp'].iloc[0]
patient_class.seizure_table['Timestamp'] - start_time

0   0 days 09:27:39.480000
1   0 days 15:17:17.480000
2   0 days 16:49:00.480000
3   0 days 18:22:05.480000
Name: Timestamp, dtype: timedelta64[ns]

In [63]:
print('SEIZURE DURATION')
for sidx in patient_class.seizure_table.index:
    try:
        end_seizure = datetime.combine(patient_class.seizure_table.loc[sidx, 'Data'], patient_class.seizure_table.loc[sidx, 'Fim'])
        print((end_seizure - patient_class.seizure_table.loc[sidx, 'Timestamp']).total_seconds())
    except:
        print('no end time')

SEIZURE DURATION
63.0
63.0
63.0
63.0


In [69]:
hr_wear.median()

hr                           101.867572
timestamp    2022-08-10 10:22:10.670432
dtype: object